# Réflexion sur l'attention
Rélexion basée sur Attention is all you need v.7 : https://arxiv.org/abs/1706.03762 (Vaswani et al.) (+http://nlp.seas.harvard.edu/annotated-transformer/)



## Cadre

L'article montre une architecture composée d'un `encodeur` et d'un `décodeur`. Prennons une traduction. L'encodeur a pour but d'apprendre une représentation dans l'espace de la source et le décodeur exécute la traduction.

## L'attention

Il existe 3 contextes de matrices d'attentions :

- Encodeur (self) : Capture les dépendances entre les mots de la phrase source en comparant chaque mot avec les autres mots de la même phrase source. Création d'une représentation riche et contextuelle.

- Décodeur (self) : A chaque étape de la génération de la phrase cible, elle compare le mot en cours de génération avec les mots précédemment générés dans la même phrase cible. Assure un alignement harmonieux.

- Décodeur (cross) Combine l'information de la phrase source et de la phrase cible en cours de génération. Le modèle de "regarde" la phrase source pendant le processus de traduction et donc s'assure de la précision et de la cohérence dans la traduction résultante.

---
NB : L'article ne couvre pas le méchanisme de tokenization mais voici la présentation d'un outil qui le fait : https://spacy.io/api et un article fondateur au niveau de la représentation de ceux-ci dans l'espace : `Efficient Estimation of Word Representations in Vector Space` https://arxiv.org/abs/1301.3781 (Mikolov et al.)

---

## Scaled dot product attention

$Attention(Q, K, V) = softmax( \frac{QK^T}{\sqrt d_k} ) V$, Où $d_k$ est la dimension de l'embedding. (2)

---

Définition de Q, K et V :

- La *requête* (query) est le mot que nous cherchons dans la séquence.
- La *clé* (key) indique ce qui est important en se basant sur la requête et peut
être vue comme une indexation unique de la valeur.
- La *valeur* (value) est l'information contenue dans le mot d'entrée, elle contribue à former la nouvelle représentation du mot en cours de génération.
- La *fonction de score* prends la requête et la clé comme entrée et sa sortie est le poids des deux (la similarité entre les deux, ce sur quoi l'attention se porte)


# Exemple

Prennons l'exemple d'une traduction. Disons que nous traduisons :
1. La branche au soleil se dore et penche pour l'abriter | $t-1$
2. ses boutons qui vont éclore sur l'oiseau qui va chanter | $t$

Une traduction anglaise serait :
1. The branch tans in the sun and lean to shelter | $t-1$
2. her pimples that are about to hatch on the bird that will sing | $t$

---

Dans attention is all you need (https://arxiv.org/abs/1706.03762) Q, K et V sont la même matrice (en pratique un tenseur avec les batchs) mais il y a aussi deux éléments intéressants dans l'architecture. C'est le shifted right en entrée du décodeur et le fait que tout soit traité en parrallèle (ce qui justifie le posisional encoding). Alors pour illustrer les concepts de multiplication matricielle, réfléchissons en décallant tout cela.

Imaginons que nous sommes au temps $t$.

Au niveau de *l'auto-attention* de l'encodeur, nous avons :

- Q = "*ses boutons qui vont éclore sur l'oiseau qui va chanter*" (sous la forme d'une matrice)
- K = "*La branche au soleil se dore et penche pour l'abriter*" (sous la forme d'une matrice)
- V = "*ses*" (sous la forme d'un vecteur dense)

Au niveau de *l'auto-attention* du décodeur, nous avons :

- Q = "*her pimples that are about to hatch on the bird that will sing*" (sous la forme d'une matrice)
- K = "*The branch tans in the sun and lean to shelter*" (sous la forme d'une matrice)
- V = "*her*" (sous la forme d'un vecteur dense)

Au niveau de *l'attention croisée* du décodeur, nous avons :


- Q = "*ses boutons qui vont éclore sur l'oiseau qui va chanter*" (sous la forme d'une matrice)
- K = "*The branch tans in the sun and lean to shelter*" (sous la forme d'une matrice)
- V = "*her*" (sous la forme d'un vecteur dense)

Sachant que les valeurs de V vaudront tour à tour chaque mot de la phrase de la requête et que tout cela est en pratique traité en parrallèle, ce que nous déconstruisons pour clarifier l'expliation.


In [61]:
import random
import numpy
import math
import warnings
from scipy.special import softmax

warnings.filterwarnings('ignore')

rand=random.Random()
dim = 512
min = 0.000001
max = 1

def generate_embedding():
  lst = []
  for i in range(dim):
    lst.append(rand.uniform(min, max))

  return lst


## Calculons un score d'attention :

Score d'attention = $Q \cdot K^T$ (1)

Reproduisons, sur base de l'exemple ..

- Q, une matrice $11 \times 512$ pour "ses boutons qui vont éclore sur l'oiseau qui va chanter"
- K, une matrice $10 \times 512$ pour "The branch tans in the sun and lean to shelter"
- V, un vecteur $1 \times 512$ pour "her"

Ce score d'attention résulte du produit d'une matrice $11 \times 512$ pour la source (query) et d'une matrice $10 \times 512$ pour la cible (key). Appliquons (1) pour obtenir une matrice $11 \times 10$ :

In [75]:
Q = [generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding()]
K = [generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding(), generate_embedding()]
V = generate_embedding()

# Affichage des dimensions
print("Q : ses boutons qui vont éclore sur l'oiseau qui va chanter : ", numpy.shape(Q))
print("K : The branch tans in the sun and lean to shelter : ", numpy.shape(K))
print()

# Transposition
K_T = (numpy.array(K).T).tolist()

# Calcul d'un score d'attention
attention_score = numpy.dot(Q, K_T)

# Affichage score d'attention
# print("score d'attention : ", attention_score)
print("dimension : ", numpy.shape(attention_score), "c'est à dire Q . K^T")
print()


Q : ses boutons qui vont éclore sur l'oiseau qui va chanter :  (11, 512)
K : The branch tans in the sun and lean to shelter :  (10, 512)

dimension :  (11, 10) c'est à dire Q . K^T



## Similarité
Au plus les valeurs du score d'attention sont élevés au plus il y a de la similarité entre les membres du produit matriciel par définition.

- Documentation géométrique : https://fr.wikipedia.org/wiki/Similitude_(g%C3%A9om%C3%A9trie)

Si la similarité est forte entre les deux matrices on dit que le modèle porte attention. Voici la tête de cette matrice dont la représentation est également générée aléatoirement :






In [67]:
# Nombre de mots dans la séquence cible (Q) et la séquence source (K)
num_words_q = len(Q)
num_words_k = len(K)

# Génération de scores de similarité aléatoires
scores = numpy.random.rand(num_words_q, num_words_k)

# Affichage de la matrice de scores
for i in range(num_words_q):
    if i > 9:
      num = str(i)
    else:
      num = "0"+str(i)

    print("mot{} (Q) |".format(num), end=" ")
    for j in range(num_words_k):
        print("{:.2f}".format(scores[i][j]), end="    ")
    print()

print("         --------------------------------------------------------------------------------")
for j in range(num_words_k):
  if j == 0:
    print("            mot{}".format(j + 1), end="   ")
  else:
    print(" mot{}".format(j + 1), end="   ")
print()

print("            Mots dans la séquence source (K)")

mot00 (Q) | 0.67    0.87    0.97    0.28    0.66    0.74    0.80    0.40    0.03    0.34    
mot01 (Q) | 0.27    0.08    0.69    0.30    0.12    0.61    0.20    0.68    0.95    0.57    
mot02 (Q) | 0.69    0.19    0.19    0.47    0.23    0.24    0.75    0.19    0.72    0.25    
mot03 (Q) | 0.22    0.87    0.23    0.17    0.16    0.62    0.50    0.71    0.36    0.53    
mot04 (Q) | 0.11    0.99    0.14    0.39    0.04    0.97    0.65    0.74    0.09    0.77    
mot05 (Q) | 0.81    0.84    0.71    0.47    0.58    0.55    0.58    0.78    0.26    0.49    
mot06 (Q) | 0.15    0.88    0.45    0.41    0.77    0.00    0.67    0.64    0.75    0.17    
mot07 (Q) | 0.06    0.19    0.63    0.01    0.22    0.12    0.02    0.28    0.38    0.54    
mot08 (Q) | 0.54    0.74    0.49    0.78    0.12    0.79    0.92    0.90    0.04    0.56    
mot09 (Q) | 0.47    0.28    0.92    0.94    0.68    0.25    0.65    0.27    0.94    0.58    
mot10 (Q) | 0.08    0.26    0.39    0.46    0.92    0.56    0.31    0.

## Calcul de l'attention

Ce qui suit consiste en
- une normalisation (mise à l'échelle) des scores
- une softmax qui transforme la matrice en un vecteur de probabilté
- un produit scalaire avec V

In [91]:
import torch
import torch.nn.functional as F

# initialization
s = []

# Mise à l'échelle
normalized_attention_score = attention_score / (math.sqrt(dim))
print("score d'attention normalisée : ")
print("------------------------------")

# Affichage de la matrice de scores
for i in range(num_words_q):
    if i > 9:
      num = str(i)
    else:
      num = "0"+str(i)

    print("mot{} (Q) |".format(num), end=" ")
    s_i = 0
    for j in range(num_words_k):
      s_i += normalized_attention_score[i][j]
      print("{:.2f}".format(normalized_attention_score[i][j]), end="    ")

    s.append(s_i)
    print()

print("         --------------------------------------------------------------------------------")
for j in range(num_words_k):
  if j == 0:
    print("            mot{}".format(j + 1), end="   ")
  else:
    print(" mot{}".format(j + 1), end="   ")
print()

print("            Mots dans la séquence source (K)")
print()

# Softmax
s_att = softmax(s)
print("probabilité d'attention : ", s_att)
print()

# Scaled dot product attention
s_att = torch.tensor(s_att)
V = torch.tensor(V)

att = torch.sum(s_att.unsqueeze(-1) * V, dim=0)

print()

print("dimensions des résultats : ")
print("--------------------------")
print()

print("s_att : ", numpy.shape(s_att))
print("V : ", numpy.shape(V))
print("dot product attention pour un mot : ", numpy.shape(att))


score d'attention normalisée : 
------------------------------
mot00 (Q) | 5.51    5.61    5.72    5.46    5.74    5.48    5.04    5.55    5.31    5.24    
mot01 (Q) | 5.74    5.76    5.69    5.70    5.82    5.61    5.25    5.55    5.38    5.46    
mot02 (Q) | 5.89    5.87    5.75    5.66    6.05    5.72    5.51    5.79    5.55    5.64    
mot03 (Q) | 5.79    5.86    5.73    5.79    5.99    5.86    5.39    5.80    5.44    5.71    
mot04 (Q) | 5.79    5.76    5.91    5.79    5.86    5.88    5.33    5.83    5.44    5.54    
mot05 (Q) | 5.86    5.93    5.87    5.90    5.98    5.76    5.33    5.92    5.57    5.62    
mot06 (Q) | 5.74    6.06    5.89    6.04    6.24    6.03    5.44    5.97    5.73    5.68    
mot07 (Q) | 5.77    5.88    6.01    5.97    5.99    5.73    5.39    5.78    5.58    5.71    
mot08 (Q) | 5.52    5.74    5.73    5.47    5.95    5.63    5.22    5.49    5.28    5.36    
mot09 (Q) | 5.49    5.62    5.73    5.47    5.64    5.52    5.26    5.47    5.38    5.25    
mot10 (

Nous avons traité le premier mot, en parrallèle sont traités chaque mots, dans l'exemple nous en avons $12$, ce qui donne une matrice $11 \times 12$ à laquelle nous ajoutons l'embedding c'est à dire un tenseur $11 \times 12 \times 512$ lorsque chaque mot est traité.

# Exercice supplémentaire

Effectuer cet exercice avec de vrais embeddings en utilisant https://fasttext.cc/